In [22]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import parsec as pc
from scipy.special import i0,i1,k0,k1
from scipy.constants import G
from scipy.optimize import curve_fit
import emcee

In [12]:
msun=1.989e+30
kpc=1e+3*pc
g1=4.302e-6

In [13]:
def vdisk(r,sigma0,rd):
    const=4*np.pi*G*sigma0*rd
    sec=i0(r/(2*rd))*k0(r/(2*rd))-i1(r/(2*rd))*k1(r/(2*rd))
    temp=1e-3*np.sqrt(np.abs(const*(r/(2*rd))**2*sec))
    return temp

In [14]:
dir="/home/esha/Documents/anurag/nav"
column_names=["Rad","Vobs","errV","Vgas","Vdisk","Vbul","SBdisk","SBbul"]
gal_data=[]
for gal in os.listdir(dir):
    
    if gal.endswith(".csv"):
        
        gal_path=os.path.join(dir,gal)
        gal_df=pd.read_csv(gal_path,names=column_names)
        gal_df['Galaxy']=os.path.splitext(gal)[0]
        gal_data.append(gal_df)
combined_df=pd.concat(gal_data,ignore_index=True)

In [15]:
for col in column_names:
    combined_df[col]=pd.to_numeric(combined_df[col])

In [16]:
uniq_gal=combined_df['Galaxy'].unique()
col=plt.cm.rainbow(np.linspace(0,1,len(uniq_gal)))

                   

In [17]:
with open('metadata.txt','r') as file:
    lines=file.readlines()
data=[]
for line in lines:
    field=line.strip().split()
    data.append(field)

    

In [18]:
Md=[]
rd=[]
for i in range(len(data)):
    if data[i][0] in uniq_gal:
        try:
            Md.append(pd.to_numeric(data[i][12]))
            rd.append(pd.to_numeric(data[i][11]))
        except ValueError:
            print("gal not found")

In [19]:
def log_likelihood(theta, r, v_obs, v_err):
    sigma0, rd = theta
    model = vdisk(r, sigma0, rd)
    sigma2 = v_err ** 2
    return -0.5 * np.sum((v_obs - model) ** 2 / sigma2 + np.log(sigma2))

def log_prior(theta):
    sigma0, rd = theta
    if 0 < sigma0 < 1e12 and 0 < rd < 1e3:  # Adjust bounds as necessary
        return 0.0
    return -np.inf

def log_probability(theta, r, v_obs, v_err):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, r, v_obs, v_err)


In [23]:
nwalkers = 32
ndim = 2
nsteps = 5000
output_file = 'galaxy_fits.txt'
with open(output_file, 'w') as f:
    f.write("Galaxy\tMd_fit\tRd_fit\n")  # Header line

    for gal in uniq_gal:
        dx = combined_df[combined_df['Galaxy'] == gal]
        r = dx['Rad'].values * kpc
        v_obs = dx['Vdisk'].values
        v_err = 0.1 * v_obs  # Assume 10% error for simplicity
        m0=(Md[uniq_gal.tolist().index(gal)]*msun)/(pc**2)
        r0=rd[uniq_gal.tolist().index(gal)]*kpc
        initial_guess=[m0,r0]

        #initial_guess = np.array([1e9, 10])  # Adjust based on expected values
        pos = initial_guess + 1e-4 * np.random.randn(nwalkers, ndim)

        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(r, v_obs, v_err))
        sampler.run_mcmc(pos, nsteps, progress=True)

        samples = sampler.get_chain(discard=100, thin=15, flat=True)

        # Plot the corner plot
        fig = corner.corner(samples, labels=["sigma0", "rd"], truths=initial_guess)
        plt.savefig(f"{gal}_corner_plot.png")
        plt.close()

        # Calculate median values and save results
        sigma0_mcmc, rd_mcmc = np.median(samples, axis=0)
        f.write(f"{gal}\t{sigma0_mcmc}\t{rd_mcmc}\n")

        # Plot the fit
        plt.figure()
        plt.scatter(r, v_obs, label='Observed Data')
        r_fit = np.linspace(min(r), max(r), 500)
        v_fit = vdisk(r_fit, sigma0_mcmc, rd_mcmc)
        plt.plot(r_fit, v_fit, label='Fitted Model', color='red')
        plt.xlabel('Radius (kpc)')
        plt.ylabel('Vdisk')
        plt.title(f'Galaxy: {gal}')
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{gal}_vdisk_fit.png")
        plt.close()


ValueError: Initial state has a large condition number. Make sure that your walkers are linearly independent for the best performance